# S4 Projcet - 사무실 일상 챗봇 <사무실 너머머> 구현

## 프로젝트 개요
1. 주제 및 데이터 소개: 자연어 처리(NLP) 활용 챗봇 만들기 
   - 데이터 : AI Hub 한국어 대화 데이터셋 - 오피스 데이터 (총 4개 파일)
2. 자연어 처리 모델: Sentence-BERT 
   - SBERT 기반 한국어 사전 학습 모델: ko-sroberta-multitask
3. 사무실 일상 챗봇 <사무실 너머머> 구현 및 배포
   - streamlit 서비스 활용 구현 및 배포  
4. 한계 및 향후 계획 


### 이 파일에서는 실제 코드 구현 부분만 담고 있습니다. 

In [1]:
# 기본 설정 
import tensorflow as tf 
import pandas as pd
import numpy as np
import json 
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import warnings

warnings.filterwarnings("ignore")
%matplotlib inline
%config InlineBackend.figure_format='retina'

# tf warning 끄기  
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

2022-12-01 21:07:02.270137: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# 코드 실행 전 seed 지정
np.random.seed(42)
tf.random.set_seed(42)

## 데이터 불러오기 및 합치기

![AI Hub 데이터 소개](./img/ai_hub_data.png "AI Hub 데이터 소개")

- 데이터 : AI Hub - 한국어 대화 데이터셋 - 오피스 데이터 (총 4개 파일)
  
- 링크 : https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&dataSetSn=272

In [3]:
# AI Hub에서 다운받은 json 데이터 불러오기 
BASE_PATH = '/Users/ju/datasets/office/'

# 1번 data (daily)
df1 = pd.read_json(BASE_PATH + 'output_daily_1st.json') 

print("\n*** df1의 데이터 크기: ", df1.shape)
print("*** df1의 domain 분포: ", df1['domain'].value_counts())

print(df1.head(2))
print("-------------------------------------------------------------")

# 2번 data (daily)
df2 = pd.read_json(BASE_PATH + 'output_daily_2nd.json') 

print("\n*** df2의 데이터 크기: ", df2.shape)
print("*** df2의 domain 분포: ", df2['domain'].value_counts())

print(df2.head(2))
print("-------------------------------------------------------------")

# 3번 data (daily)
df3 = pd.read_json(BASE_PATH + 'output_daily_3rd.json') 

print("\n*** df3의 데이터 크기: ", df3.shape)
print("*** df3의 domain 분포: ", df3['domain'].value_counts())

print(df3.head(2))


*** df1의 데이터 크기:  (7427, 4)
*** df1의 domain 분포:  daily    7427
Name: domain, dtype: int64
   index domain user_utterance        system_utterance
0      0  daily           null  이영자 선임님. 곧 퇴근하실 시간입니다.
1      0  daily         알고 있어.        오늘도 고생 많이 하셨습니다.
-------------------------------------------------------------

*** df2의 데이터 크기:  (5726, 4)
*** df2의 domain 분포:  daily    5726
Name: domain, dtype: int64
   index domain user_utterance system_utterance
0      0  daily           null           안녕하세요.
1      0  daily            안녕.    아침은 드시고 오셨나요?
-------------------------------------------------------------

*** df3의 데이터 크기:  (2814, 4)
*** df3의 domain 분포:  daily    2814
Name: domain, dtype: int64
   index domain   user_utterance           system_utterance
0      0  daily             null           정혜원 대표님, 안녕하세요. 
1      0  daily  안녕하세요, 자주 만나네요.  비도 오는데 먼걸음 하시느라 고생하셨습니다. 


In [4]:
# 4번 data (목적성 대화: meeting, email. schedule)
df4 = pd.read_json(BASE_PATH + 'output_task.json') 

print("\n*** df3의 데이터 크기: ", df4.shape)
print("*** df3의 domain 분포: ", df4['domain'].value_counts())

df4.head()


*** df3의 데이터 크기:  (30447, 4)
*** df3의 domain 분포:  meeting     22726
schedule     3727
email        2956
             1038
Name: domain, dtype: int64


,index,domain,user_utterance,system_utterance
0,0,meeting,null,"김철수 센터장님, 안녕하세요?"
1,0,meeting,null,오늘 오후 2시에 인공지능연구센터 중회의실에서 컴패니언 대화엔진 관련 팀회의가 있습...
2,0,meeting,null,참석가능하신가요?
3,0,meeting,참석할게.,"네, 알겠습니다."
4,1,meeting,null,"김철수 센터장님, 안녕하세요?"


In [5]:
# 데이터 합치기 
df = pd.concat([df1, df2, df3, df4])

print("\n*** df의 데이터 크기: ", df.shape)
print("*** df의 domain 분포: ", df['domain'].value_counts())

df.head()


*** df의 데이터 크기:  (46414, 4)
*** df의 domain 분포:  meeting     22726
daily       15967
schedule     3727
email        2956
             1038
Name: domain, dtype: int64


,index,domain,user_utterance,system_utterance
0,0,daily,null,이영자 선임님. 곧 퇴근하실 시간입니다.
1,0,daily,알고 있어.,오늘도 고생 많이 하셨습니다.
2,1,daily,null,이영자 선임님. 곧 퇴근하실 시간입니다.
3,1,daily,밖에 날씨는?,월요일 퇴근전 현재 맑고 저온건조한데다가 공기는 맑은 편입니다.
4,1,daily,알았다.,오늘도 고생이 많으셨습니다.


## 데이터 전처리

In [6]:
# 결측치 확인 
print(df.isnull().sum())

# 'null' -> NaN 값으로 변경 
df['user_utterance'] = df['user_utterance'].replace('null', np.nan)
df['system_utterance'] = df['system_utterance'].replace('null', np.nan)

# 결측치 재확인 
print(df.isnull().sum())
df.head()

index               0
domain              0
user_utterance      0
system_utterance    0
dtype: int64
index                   0
domain                  0
user_utterance       9148
system_utterance    10618
dtype: int64


,index,domain,user_utterance,system_utterance
0,0,daily,NaN,이영자 선임님. 곧 퇴근하실 시간입니다.
1,0,daily,알고 있어.,오늘도 고생 많이 하셨습니다.
2,1,daily,NaN,이영자 선임님. 곧 퇴근하실 시간입니다.
3,1,daily,밖에 날씨는?,월요일 퇴근전 현재 맑고 저온건조한데다가 공기는 맑은 편입니다.
4,1,daily,알았다.,오늘도 고생이 많으셨습니다.


In [7]:
# 결측치 제거 -> 다시 확인 
df = df.dropna()
df.isnull().sum()

index               0
domain              0
user_utterance      0
system_utterance    0
dtype: int64

In [8]:
# 중복값 확인 -> 확인 
df.duplicated().sum()

0

In [9]:
# 직관적으로 이해하기 위해 컬럼명 'qeustion', 'answer' 방식으로 변경 
df = df.rename(columns = {'user_utterance' : 'question', 'system_utterance': 'answer'})
df.head()

,index,domain,question,answer
1,0,daily,알고 있어.,오늘도 고생 많이 하셨습니다.
3,1,daily,밖에 날씨는?,월요일 퇴근전 현재 맑고 저온건조한데다가 공기는 맑은 편입니다.
4,1,daily,알았다.,오늘도 고생이 많으셨습니다.
5,2,daily,퇴근한다.,오늘도 일하느라 수고하셨어요.
6,2,daily,알면 됐다.,그럼 내일 뵙겠습니다.


In [10]:
# 불필요한 컬럼 삭제
df = df[['domain', 'question', 'answer']]
df.head()

,domain,question,answer
1,daily,알고 있어.,오늘도 고생 많이 하셨습니다.
3,daily,밖에 날씨는?,월요일 퇴근전 현재 맑고 저온건조한데다가 공기는 맑은 편입니다.
4,daily,알았다.,오늘도 고생이 많으셨습니다.
5,daily,퇴근한다.,오늘도 일하느라 수고하셨어요.
6,daily,알면 됐다.,그럼 내일 뵙겠습니다.


In [16]:
# 인덱스 재지정 
df = df.reset_index(drop = True)
df.head()

,domain,question,answer
0,daily,알고 있어.,오늘도 고생 많이 하셨습니다.
1,daily,밖에 날씨는?,월요일 퇴근전 현재 맑고 저온건조한데다가 공기는 맑은 편입니다.
2,daily,알았다.,오늘도 고생이 많으셨습니다.
3,daily,퇴근한다.,오늘도 일하느라 수고하셨어요.
4,daily,알면 됐다.,그럼 내일 뵙겠습니다.


In [20]:
# 최종 데이터 크기 
print(df.shape)

# domain 분포 
df['domain'].value_counts()

(26648, 3)


daily       13611
meeting      6893
email        2924
schedule     2784
              436
Name: domain, dtype: int64

In [11]:
# 재사용을 csv로 저장 
df.to_csv('office_conversation.csv')

## 자연어 처리 모델: Sentence-BERT 모델 로드

![SBERT 논문](./img/SBERT_paper.png "SBERT 논문")

- **Sentence-BERT**: 대표적인 ‘언어 모델(language model)’인 BERT 모델(Google, 2018년 발표)에 미세학습조정 (fine-tuning)을 통해 문장 임베딩의 ‘성능’과 ‘속도’를 개선시킨 모델. 문장의 의미적 유사도 검색 등에 강점을 지니고 있음. 

- 참고논문 : Reimers, Nils, and Iryna Gurevych. "Sentence-bert: Sentence embeddings using siamese bert-networks." arXiv preprint arXiv:1908.10084 (2019).
  
- 링크 : https://arxiv.org/pdf/1908.10084.pdf

### SBERT 기반 한국어 사전 학습 모델: ko-sroberta-multitask

![SBERT 기반 한국어 사전 학습 모델](./img/hugging_ko_sroberta.png "SBERT 기반 한국어 사전 학습 모델")

- 링크: (허깅페이스) https://huggingface.co/jhgan/ko-sroberta-multitask

In [12]:
# 사전 학습된 SBERT 기반 한국어 모델 불러오기 
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

# 예제로 임베딩 확인 
sentences = ["안녕하세요?", "한국어 문장 임베딩을 위한 버트 모델입니다."] # 768 차원의 벡터로 바뀜
embeddings = model.encode(sentences)

print(embeddings)

[[-0.37510464 -0.7733839   0.5927711  ...  0.57923526  0.32683483
  -0.6508965 ]
 [-0.09361704 -0.18191524 -0.19230816 ... -0.03165802  0.30412534
  -0.2679362 ]]


In [14]:
# 임베딩 크기 (문장 수, 768 차원) 확인
embeddings.shape

(2, 768)

## 챗봇 데이터에 문장 임베딩 적용

In [17]:
df.loc[0, 'question']

'알고 있어.'

In [18]:
# 첫번째 문장 -> 임베딩 확인 
model.encode(df.loc[0, 'question'])

array([ 1.39510900e-01, -7.23620176e-01, -3.65751863e-01,  1.49114162e-01,
       -3.20672244e-01, -2.50401258e-01, -4.96178716e-01,  1.16734743e-01,
        1.83035776e-01,  9.19244587e-02, -3.74977827e-01, -3.82542968e-01,
        1.29345655e-02,  4.20451164e-01,  5.10449111e-01, -2.47830376e-01,
        1.47755474e-01,  1.32183820e-01,  4.50318724e-01, -7.48629153e-01,
        1.41782209e-01,  3.17309976e-01,  3.85159217e-02,  4.20365810e-01,
        1.69757679e-01,  1.52960911e-01, -4.37234640e-02, -1.97519436e-02,
        8.24200928e-01, -9.44392085e-02, -3.43870483e-02, -3.46485257e-01,
       -1.89879999e-01, -4.73274708e-01,  5.06073534e-01,  2.04955459e-01,
        7.12405443e-01, -1.55949295e-01, -3.51359218e-01, -4.70389694e-01,
       -4.59905975e-02, -3.29444736e-01, -5.12373388e-01,  2.11201683e-01,
       -2.43167937e-01, -7.13101149e-01,  2.15692043e-01, -2.81167179e-01,
       -4.20595497e-01, -4.68152724e-02,  5.21526039e-02, -9.59128618e-01,
       -3.15538198e-01,  

In [8]:
df['embedding'] = pd.Series([[]] * len(df)) 

df['embedding'] = df['question'].map(lambda x: list(model.encode(x)))

df.head()

,domain,question,answer,embedding
0,daily,알고 있어.,오늘도 고생 많이 하셨습니다.,"[0.1395109, -0.7236202, -0.36575186, 0.1491141..."
1,daily,밖에 날씨는?,월요일 퇴근전 현재 맑고 저온건조한데다가 공기는 맑은 편입니다.,"[-0.6533351, -0.36782613, 0.78354526, -0.02236..."
2,daily,알았다.,오늘도 고생이 많으셨습니다.,"[0.08001693, -0.5358173, 0.012687151, -0.01290..."
3,daily,퇴근한다.,오늘도 일하느라 수고하셨어요.,"[-0.17582458, 0.32981366, -0.2852519, -0.15695..."
4,daily,알면 됐다.,그럼 내일 뵙겠습니다.,"[-0.018741427, -0.35529342, 0.28030354, 0.0929..."


In [9]:
df.to_csv('office_conversation_final.csv', index=False)

## 간단한 챗봇 테스트

In [10]:
text = '좋은 아침입니다.'

embedding = model.encode(text)

In [11]:
df['distance'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

df.head()

,domain,question,answer,embedding,distance
0,daily,알고 있어.,오늘도 고생 많이 하셨습니다.,"[0.1395109, -0.7236202, -0.36575186, 0.1491141...",0.306625
1,daily,밖에 날씨는?,월요일 퇴근전 현재 맑고 저온건조한데다가 공기는 맑은 편입니다.,"[-0.6533351, -0.36782613, 0.78354526, -0.02236...",0.378197
2,daily,알았다.,오늘도 고생이 많으셨습니다.,"[0.08001693, -0.5358173, 0.012687151, -0.01290...",0.299849
3,daily,퇴근한다.,오늘도 일하느라 수고하셨어요.,"[-0.17582458, 0.32981366, -0.2852519, -0.15695...",0.283873
4,daily,알면 됐다.,그럼 내일 뵙겠습니다.,"[-0.018741427, -0.35529342, 0.28030354, 0.0929...",0.273965


In [12]:
# 질문과 유사한 문장 검색 후 -> 답변 반환    
answer = df.loc[df['distance'].idxmax()]

print('구분', answer['domain'])
print('유사한 질문', answer['question'])
print('챗봇 답변', answer['answer'])
print('유사도', answer['distance'])

구분 daily
유사한 질문 좋은 아침이야. 
챗봇 답변 오늘 기분 좋은 일 있으세요? 
유사도 0.9867659211158752


## 챗봇 구현 및 배포를 위한 streamlit 실행 파일 작성

- Streamlit(https://streamlit.io/) 서비스 활용
  
- chatbot.py에 별도 구현 및 실행에 활용

In [ ]:
import streamlit as st
from streamlit_chat import message
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import json

@st.cache(allow_output_mutation=True)
def cached_model():
    model = SentenceTransformer('jhgan/ko-sroberta-multitask')
    return model

@st.cache(allow_output_mutation=True)
def get_dataset():
    df = pd.read_csv('./office_conversation_final.csv')
    df['embedding'] = df['embedding'].apply(json.loads)
    return df

model = cached_model()
df = get_dataset()

st.header('Kong\'s 사무실 일상 챗봇 <사무실 너머머>')
st.markdown("[제작: Kong Ju](https://kongju7.github.io)")

if 'generated' not in st.session_state:
    st.session_state['generated'] = []

if 'past' not in st.session_state:
    st.session_state['past'] = []

with st.form('form', clear_on_submit=True):
    user_input = st.text_input('당신: ', '')
    submitted = st.form_submit_button('전송')

if submitted and user_input:
    embedding = model.encode(user_input)

    df['distance'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())
    answer = df.loc[df['distance'].idxmax()]

    st.session_state.past.append(user_input)
    st.session_state.generated.append(answer['answer'])

for i in range(len(st.session_state['past'])):
    message(st.session_state['past'][i], is_user=True, key=str(i) + '_user')
    if len(st.session_state['generated']) > i:
        message(st.session_state['generated'][i], key=str(i) + '_bot')


![챗봇](./img/chatbot.png "챗봇")

- 링크: https://kongju7-my-project4-chatbot-63c0p0.streamlit.app/